### Buffer FAO Names hydrobasins

* Purpose of script: Buffer FAO Names hydrobasins to avoid assigning names to basins that only slightly touch other polygons
* Author: Rutger Hofste
* Kernel used: python35
* Date created: 20170823

https://gist.github.com/tmcw/3987659

In [18]:
S3_INPUT_PATH = "s3://wri-projects/Aqueduct30/processData/Y2017M08D23_RH_Merge_FAONames_V01/output/"
S3_OUTPUT_PATH = "s3://wri-projects/Aqueduct30/processData/Y2017M08D23_RH_Buffer_FAONames_V01/output/"
EC2_INPUT_PATH = "/volumes/data/Y2017M08D23_RH_Buffer_FAONames_V01/input/"
EC2_OUTPUT_PATH = "/volumes/data/Y2017M08D23_RH_Buffer_FAONames_V01/output/"
INPUT_FILE_NAME = "hydrobasins_fao_fiona_merged_v01.shp"
OUTPUT_FILE_NAME = "hydrobasins_fao_fiona_merged_buffered_v01.shp"
OUTPUT_FILE_NAME_PROJ = "hydrobasins_fao_fiona_merged_buffered_v01_WTF.prj"
BUFFERDIST = -0.005 # Buffer distance in Degrees 

In [2]:
!mkdir -p {EC2_INPUT_PATH}
!mkdir -p {EC2_OUTPUT_PATH}

In [3]:
!aws s3 cp {S3_INPUT_PATH} {EC2_INPUT_PATH} --recursive --quiet

In [11]:
from osgeo import gdal,ogr,osr
import os

In [5]:
inputLocation = os.path.join(EC2_INPUT_PATH,INPUT_FILE_NAME)
outputLocation = os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILE_NAME)

In [6]:
os.environ['SHAPE_ENCODING'] = "utf-8"

In [7]:
def createBuffer(inputfn, outputBufferfn, bufferDist):
    inputds = ogr.Open(inputfn)
    inputlyr = inputds.GetLayer()
    featureCount = inputlyr.GetFeatureCount()
    print(featureCount)

    shpdriver = ogr.GetDriverByName('ESRI Shapefile')
    if os.path.exists(outputBufferfn):
        shpdriver.DeleteDataSource(outputBufferfn)
    outputBufferds = shpdriver.CreateDataSource(outputBufferfn)
    bufferlyr = outputBufferds.CreateLayer(outputBufferfn, geom_type=ogr.wkbPolygon)
    featureDefn = bufferlyr.GetLayerDefn()
    
    i = 0
    for feature in inputlyr:
        if i % 500 ==0:
            print("Percentage Completed: %0.2d"% ((i/featureCount)*100))
        i += 1
            
        ingeom = feature.GetGeometryRef()
        geomBuffer = ingeom.Buffer(bufferDist)

        outFeature = ogr.Feature(featureDefn)
        outFeature.SetGeometry(geomBuffer)
        bufferlyr.CreateFeature(outFeature)
        outFeature = None

In [8]:
createBuffer(inputLocation, outputLocation, BUFFERDIST)

33866
Percentage Completed: 00
Percentage Completed: 01
Percentage Completed: 02
Percentage Completed: 04
Percentage Completed: 05
Percentage Completed: 07
Percentage Completed: 08
Percentage Completed: 10
Percentage Completed: 11
Percentage Completed: 13
Percentage Completed: 14
Percentage Completed: 16
Percentage Completed: 17
Percentage Completed: 19
Percentage Completed: 20
Percentage Completed: 22
Percentage Completed: 23
Percentage Completed: 25
Percentage Completed: 26
Percentage Completed: 28
Percentage Completed: 29
Percentage Completed: 31
Percentage Completed: 32
Percentage Completed: 33
Percentage Completed: 35
Percentage Completed: 36
Percentage Completed: 38
Percentage Completed: 39
Percentage Completed: 41
Percentage Completed: 42
Percentage Completed: 44
Percentage Completed: 45
Percentage Completed: 47
Percentage Completed: 48
Percentage Completed: 50
Percentage Completed: 51
Percentage Completed: 53
Percentage Completed: 54
Percentage Completed: 56
Percentage Complete

In [12]:
spatialRef = osr.SpatialReference()
spatialRef.ImportFromEPSG(4326)

6

In [19]:
outputLocationProj = os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILE_NAME_PROJ)

In [22]:
spatialRef = osr.SpatialReference()
spatialRef.ImportFromEPSG(26912)

spatialRef.MorphToESRI()
file = open(outputLocationProj, 'w')
file.write(spatialRef.ExportToWkt())
file.close()

In [21]:
!aws s3 cp {EC2_OUTPUT_PATH} {S3_OUTPUT_PATH} --recursive --quiet

In [24]:
print(GDAL_DATA)

NameError: name 'GDAL_DATA' is not defined